# 说明： 

模型在训练过程中(model.fit)，如果没有一种对模型进行相对实时的保存的话，万一停电或内存不足就麻烦了！
—— 保存了的模型（不管保存的是当前权重参数，还是整个模型），再次加载进来后就可以继续训练。

本节所使用的，用于在训练过程中实时保存模型的函数：
- tensorflow下：tf.keras.callbacks.ModelCheckpoint
- keras下：keras.callbacks.ModelCheckpoint

函数里面的“关键”参数说明：每个epoch保存一次
- filepath：模型文件保存的路径；可以预先写好一个字符串；
- monitor = 'val_loss'：网络所实时监控的变量，默认是val_loss
- save_best_only = False：是否只保存最好的模型，监控/衡量的是上面monitor的东西；默认为False
- save_weights_only = False：是否只保存的是“模型参数”；默认为False，即保存整个模型

# 用20文件中“CNN分类MNIST手写体”为例子：

In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
import keras.datasets.mnist as mnist
(train_image, train_label), (test_image, test_label) = mnist.load_data() 

In [3]:
train_image = np.expand_dims(train_image, axis = -1)
test_image = np.expand_dims(test_image, axis = -1)

In [4]:
model = keras.Sequential()

In [5]:
from keras import layers

In [6]:
# 第一层要给输入数据的形状：只要管最后3个维度，前面的batch维不用管
model.add( layers.Conv2D( filters=64, kernel_size=(3,3), activation = 'relu', input_shape=(28,28,1) ) )  # 其他一般都用默认
model.add( layers.Conv2D( filters=64, kernel_size=(3,3), activation='relu') )
model.add( layers.MaxPooling2D()  )  # 池化层一般都用默认的

In [7]:
# 进入全连接层：
model.add( layers.Flatten() )  # 把(12,12,64)全部展平为12*12*64 = 9216 —— 前面已经说过这个三维数据里都是特征！！！
model.add(layers.Dense(256, activation='relu'))
model.add( layers.Dropout(0.5) )  # 网络容量还是有些大，dropout一下
model.add(layers.Dense(10, activation='softmax'))  # 最后是10分类输出，激活用softmax多分类

In [8]:
model.compile( optimizer='adam',
               loss = 'sparse_categorical_crossentropy',  # 顺序编码
               metrics=['acc']
)

### 回调函数的设置： 

In [12]:
# 设置保存位置：直接保存整个模型 —— 每一次生成，都会把上一个文件替换掉
checkpoint_path = 'E:/Python_code/keras_total/回调函数保存/gby.h5'

# 回调函数初始化：
cp_callback = keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only = False)   # 保存整个模型，每个epoch都保存（一直覆盖）
cp_callback = keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only = True)    # 只保存最好的那个模型（自动监控）

In [13]:
# 开始训练：
# callbacks = [] 是一个列表，说明可以同时用多个回调函数
model.fit(train_image, train_label, epochs = 3, batch_size = 512, validation_data=(test_image, test_label), callbacks=[cp_callback] )

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 132s 2ms/step - loss: 0.0987 - acc: 0.9700 - val_loss: 0.0507 - val_acc: 0.9826
Epoch 2/2
60000/60000 [==============================] - 128s 2ms/step - loss: 0.0670 - acc: 0.9795 - val_loss: 0.0462 - val_acc: 0.9836
